In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

모델 평가

오버피팅(overfitting)과 언더피팅(underfitting)  

오버피팅은 특정 데이터 셋에 과도하게 적합된 것을 의미한다. 오버피팅이 발생하는 경우, 얼핏 정확도가 높아 보이지만 특정 데이터 셋에만 적합되어 알려지지 않은 데이터에 대한 예측력은 낮아지게 된다. 언더피팅은 데이터 셋에 적합이 잘되지 않은, 즉 과소적합된 것을 의미한다.  
머신러닝을 통해 모델을 학습하는 이유는 데이터의 종류와 상관없이 일반화할 수 있는 모델을 생성하는 것이다. 주어진 데이터 셋에 대해 오버피팅이나 언더피팅이 발생한다면 새로운 데이터에 적용할 수 있는 좋은 모델이라고 말하기 어렵다.  
오버피팅된 모델을 새로운 데이터셋에 적용한다면 학습 데이터 셋과는 큰 오차를 보인다. 반대로 언더피팅은 학습 데이터 셋과 테스트 데이터 셋 모두 큰 오차를 보인다.

편향-분산 트레이드오프(bias-variance tradeoff)  

편향 분산 트레이드오프란, 편향이 낮을수록 분산은 커지고, 반대로 편향이 높을수록 분산이 작아지는 경향이 있다는 것을 의미한다.  
분산이 높은 현상은 주로 복잡한 모델에 나타나고 모델이 복잡하다는 말은 오버피팅이 발생할 가능성이 높다는 뜻이다. 즉, 복잡한 모델일수록 오버피팅이 발생할 가능성이 높으며, 이는 분산이 커진다는 것을 의미한다.  
반대로 편향이 큰 현상은 주로 간단한 모델일 때 나타나는데 모델이 간단하다는 말은 언더피팅이 발생할 가능성이 높다는 뜻이다. 간단한 모델일수록 언더피팅이 발생할 가능성이 높으며, 이에 따라 편향이 커질 수 있다.

크로스 벨리데이션(cross validation, 교차 검증)  

모델을 생성한 후 실제 데이터에 적용해 보고 성능을 평가해야 하는데, 데이터 셋 전체를 학습에 사용하면 새롭게 적용할 데이터가 없어서 문제가 발생되기 때문에 전체 데이터를 학습 데이터와 테스트 데이터로 분할해서 사용한다. 학습 데이터는 학습하는 데 사용되고, 테스트 데이터는 학습 시에는 사용되지 않고 모델의 성능을 평가할 때 사용한다.  
머신러닝 알고리즘을 적용할 때 다양한 하이퍼파라미터에 대해 여러 가지 모델 후보군을 생성하고 평가한 후 최종 모델을 선택하게 된다. 이때, 파라미터는 모델 내부에서 데이터에 의해 추정되는 값이고, 하이퍼파라미터는 사용자가 직접 정하는 값이다.  
하이퍼파라미터를 결정하는 과정에서 학습 데이터와 테스트 데이터만 존재한다면 테스트 데이터에 의해 최종 모델의 파라미터가 결정된다. 즉, 모델의 하이퍼파라미터가 테스트 데이터에 의존한다는 뜻이다.  
이 문제를 해결하기 위해 학습 데이터의 일부를 검증 데이터로 사용한다. 즉, 학습 데이터는 파라미터를 구하는데 사용하고, 검중 데이터는 하이퍼파라미터를 정하는데 사용한다.  
주어진 데이터셋에 대해서 학습 데이터, 검증 데이터, 테스트 데이터로 분할할 수 있는 다양한 조합 방법이 존재한다. 이처럼 다양한 조합을 통해 모델의 성능을 검증하는 것을 크로스-밸리데이션이라고 한다.

In [2]:
from sklearn import datasets                         # 사이킷런이 제공하는 데이터 셋을 사용하기 위해 import 한다.
from sklearn.model_selection import train_test_split # 학습 데이터와 테스트 데이터를 나누기 위해 import 한다.
from sklearn.preprocessing import StandardScaler     # 표준화 스케일링을 하기 위해 import 한다.
from sklearn.linear_model import LinearRegression    # 사이킷런이 제공하는 선형 회귀 모델을 사용하기 위해 import 한다.
from sklearn.neighbors import KNeighborsClassifier   # 사이킷런이 제공하는 최근접 이웃 모델을 사용하기 위해 import 한다.
from sklearn.pipeline import Pipeline                # 파이프라인을 사용하기 위해 import 한다.

from sklearn.metrics import mean_absolute_error      # 평균 절대값 오차(MAE)를 계산하기 위해 import 한다.
from sklearn.metrics import mean_squared_error       # 평균 제곱 오차(MSE)를 계산하기 위해 import 한다.
from sklearn.metrics import accuracy_score           # 정확도를 계산하기 위해 import 한다.
from sklearn.metrics import confusion_matrix         # 혼동 행렬을 계산하기 위해 import 한다.
from sklearn.metrics import classification_report    # 분류 리포트를 출력하기 위해 import 한다.
from sklearn.metrics import r2_score                 # R 제곱값을 계산하기 위해 import 한다.
from sklearn.metrics import silhouette_score         # 실루엣 스코어를 계산하기 위해 import 한다.

In [3]:
# 사이킷런이 제공하는 보스턴 집값 데이터 셋에서 피쳐와 레이블 데이터 읽기
raw_data = datasets.load_boston()
xData = raw_data.data # 데이터
yData = raw_data.target # 데이터에 따른 레이블
print(xData.shape, yData.shape)

(506, 13) (506,)


파이프라인  

머신러닝 과정에서 파이프라인을 사용하면 데이터 전처리와 학습 모델을 연결해 코드를 간결화 할 수 있다.

파이프라인을 사용하지 않은 코드

In [4]:
# 학습 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=7)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# 표준화 스케일링 적용
std_scale = StandardScaler()
x_train = std_scale.fit_transform(x_train) # 학습 데이터 스케일링
x_test = std_scale.transform(x_test) # 테스트 데이터 스케일링
# 모델 생성 후 학습
clf = LinearRegression() # 선형 회귀 모델을 만든다.
clf.fit(x_train, y_train) # 스케일링된 학습 데이터와 학습 데이터에 따른 레이블로 학습시킨다.
# 모델 예측
predict = clf.predict(x_test) # 스케일링된 테스트 데이터로 학습된 모델을 예측한다.
# 모델 평가
# 테스트 데이터의 레이블과 테스트 데이터로 예측한 값의 평균 제곱 오차를 계산한다.
print(mean_squared_error(y_test, predict))
# 테스트 데이터의 레이블과 테스트 데이터로 예측한 값의 평균 절대값 오차를 계산한다.
print(mean_absolute_error(y_test, predict))

(379, 13) (127, 13) (379,) (127,)
29.51513779019758
3.5678438946275453


파이프라인을 사용하는 코드

In [5]:
# 학습 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=7)

# 파이프라인 => 표준화 스케일링(전처리) + 학습 모델 객체 생성
pipeline = Pipeline(
    [
        ('scaler', StandardScaler()), # 표준화 스케일링 적용
        ('linear_regression', LinearRegression()) # 선형 회귀 모델 적용
    ]
)

# 파이프라인에서 스케일러와 모델 생성 후 학습
pipeline.fit(x_train, y_train)
# 모델 예측
predict = pipeline.predict(x_test)
# 모델 평가
print(mean_squared_error(y_test, predict))
print(mean_absolute_error(y_test, predict))

29.51513779019758
3.5678438946275453


그리드 서치(Grid Search)

머신러닝 과정에서 관심있는 매개 변수들을 대상으로 학습 가능하도록 만드는 방식을 말한다.

k-최근접 이웃 알고리즘 사용시 1부터 10 사이의 k값 후보 중 가장 높은 성능을 보이는 k값 정하기

In [6]:
# 사이킷런이 제공하는 붓꽃 셋에서 피쳐와 레이블 데이터 읽기
raw_data = datasets.load_iris()
xData = raw_data.data
yData = raw_data.target
print(xData.shape, yData.shape)

(150, 4) (150,)


In [7]:
# 학습 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# 표준화 스케일링 적용
std_scale = StandardScaler()
x_train = std_scale.fit_transform(x_train)
x_test = std_scale.transform(x_test)

best_accuracy = 0 # 최고 정확도를 기억할 변수를 선언하고 0으로 초기화 한다.
# 1부터 10 사이의 k값 후보 중 가장 높은 성능을 보이는 k값을 찾는다.
for k in range(1, 11):
    # n_neighbors 인수로 k값을 넘겨서 최근접 이웃 모델을 만든다.
    clf_knn = KNeighborsClassifier(n_neighbors=k)
    # 학습 데이터와 학습 데이터에 따른 레이블을 넘겨서 최근접 이웃 모델을 학습시킨다.
    clf_knn.fit(x_train, y_train)
    # 학습된 모델에 테스트 데이터를 넘겨서 예측한다.
    predict = clf_knn.predict(x_test)
    # 테스트 데이터의 레이블과 예측값을 넘겨서 정확도를 계산한다.
    accuracy = accuracy_score(y_test, predict)
    print('k: {:2d}, accuracy: {:6.3f}'.format(k, accuracy))
    
    # 가장 정확도가 높은 k를 계산한다.
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        final_k = k
    # ===== if
# ===== for
print('best k: {:2d}, accuracy: {:6.3f}'.format(final_k, accuracy))

(112, 4) (38, 4) (112,) (38,)
k:  1, accuracy:  0.921
k:  2, accuracy:  0.947
k:  3, accuracy:  0.974
k:  4, accuracy:  0.974
k:  5, accuracy:  0.974
k:  6, accuracy:  0.974
k:  7, accuracy:  0.974
k:  8, accuracy:  0.974
k:  9, accuracy:  0.974
k: 10, accuracy:  0.974
best k:  3, accuracy:  0.974


손실 함수(loss function)와 비용 함수(cost function)의 개념

손실 함수는 머신러닝을 통해 생성한 모델이 실제값과 얼마나 차이가 나는지 즉, 손실 정도를 나타내는 함수로 모델 손실은 예측값과 실제값의 차이를 이용해 측정한다.  
손실 함수와 비슷하게 비용 함수라는 개념도 존재하는데 손실 함수는 각 데이터 포인트에 대해서 예측값과 실제값의 차이를 나타내지만, 비용 함수는 데이터 셋 전체를 대상으로 하는 손실이다. 엄밀하게 말하면 서로 다르다고 한 수도 있으나 실제로는 손실 함수와 비용 함수를 구분없이 사용하기도 한다.

손실 함수는 크게 L1 손실(L1 loss)과 L2 손실(L2 loss)이 존재한다.

L1 손실 함수  
L1 손실은 다른말로 L1 비용(L1 cost)이라고도 부르며, 아래와 같이 표현된다.

$$L1 = \sum |y_{true} - y_{predict}|$$

$y_{true}$는 실제값을 의미하고 y_{predict}는 학습 모델을 이용해 예측한 값을 의미한다.  
즉, L1 손실은 실제값과 예측값의 차이에 절대값을 취한것으로 실제값과 예측값의 차이를 줄이는 것이 목적이다.

L1 손실과 관련된 비용 함수로 MAE(Mean Absolute Error)가 있다.

$$MAE = \frac {1}{n} \sum_{i=1}^{n} |y_i - \hat y|$$

L2 손실 함수  
L2 손실은 실제값과 예측값의 차이에 제곱을 취함으로써 구할 수 있다.

$$L2 = \sum (y_{true} - y_{predict})^2$$

L2 손실을 이용한 비용 함수에는 MSE(Mean Squared Error), RMSE(Root Mean Squared Error)가 존재한다.

$$MSE = \frac {1}{n} \sum_{i=1}^{n} (y_i - \hat y)^2, \; RMSE = \sqrt{MSE}$$

MSE는 흔히 사용하는 비용 함수로 실제값과 예측값의 차이의 제곱의 평균을 의미하고 RMSE는 MSE에 제곱근을 취한 형태이다.  
MSE를 구하는 과정에서 실제값과 예측값의 차이를 제곱하므로 MSE는 이상치(outlier)의 변화에 민감하다. 반면에 MAE나 RMSE는 이상치와 상관없이 안정된 값을 보여준다. 머신러닝에서 이상치에 중점을 두고 싶다면 MSE를 사용하고, 그렇치 않다면 MAE나 RMSE를 사용할 수 있다.

엔트로피(Entropy)

엔트로피는 정보 이론에서 사용하는 개념으로 확률 변수의 불확실성 정도를 측정하기 위해 사용한다.

$$Entropy(P) = - \sum P(x)logP(x) = -E(logP(x))$$

위 엔트로피 식은 Entropy(P)로 표시했지만 H(P) 혹은 H(X)라고 쓰기도 하며 엔트로피는 의사결정 트리에서 주로 사용한다.

크로스 엔트로피

$$Cross Entropy(P, Q) = - \sum P(x)logQ(x) = -E_p(logQ(x))$$

위 식은 크로스 엔트로피라고 하는데, 크로스 엔트로피는 하나의 분포를 대상으로 하는 반면, 크로스 엔트로피는 두 분포 P(x), Q(x)를 대상으로 엔트로피를 측정해서 두 분포간의 차를 계산한다. 머신러닝에서 크로스 엔트로피를 사용할 때는 P(x)를 실제 모형의 분포, Q(x)를 추정 모형의 분포라고 설정한다.

쿨백-라이블러 발산

$$
\begin{aligned}
D_{KL}(P||Q) =& \sum P(x)log \frac{P(x)}{Q(x)} \\
=& -\sum P(x)logQ(x) + \sum P(x)logP(x) \\
=& -E_p(log \frac{P(x)}{Q(x)})
\end{aligned}
$$

위 식은 쿨백-라이블러 발산(KLD)이라는 개념이다. 크로스 엔트로피와 쿨백-라이블러 발산은 머신러닝에서 자주 사용되는 손실 함수이다. 쿨백-라이블러는 다른 말로 상대적 엔트로피(Relative Entropy)라고도 부른다.

모델 성능 평가에 필요한 개념

True(정답)으로 분류되는 경우  
데이터를 양성(Positive)으로 예측했을 때, 실제값도 양성일 경우 => TP(True Positive)  
데이터를 음성(Negative)으로 예측했을 때, 실제값도 음성일 경우 => TN(True Negative)  

False(오답)으로 분류되는 경우  
데이터를 양성으로 예측했을 때, 실제값이 음성일 경우 => FP(False Positive)  
데이터를 음성으로 예측했을 때, 실제값이 양성일 경우 => FN(False Negative)

모델 성능 평가 지표

정확도(Accuracy): 전체 데이터 중 양성으로 분류되는 비율 => $\frac{TP+TN}{TP+TN+FP+FN}$  
에러율(Error Rate): 전체 데이터 중 음성으로 분류되는 비율 => $\frac{FP+FN}{TP+TN+FP+FN}$  

정밀도(Precision): 양성으로 예측했을 때, 실제 양성으로 나타나는 비율 => $\frac{TP}{TP+FP}$  
특이도(Specificity): 음성으로 예측했을 때, 실제 음성으로 나타나는 비율 => $\frac{TN}{TN+FN}$  

재현율(Recall): 실제 양성에 해당되는 데이터가 양성으로 나타나는 비율 => $\frac{TP}{TP+FN}$  
FPR(False Positive Rate): 실제 음성에 해당되는 데이터가 양성으로 나타나는 비율 => $\frac{FP}{TP+FN}$  

f1 score: 정밀도와 재현율의 조화 평균값이다. 0부터 1 사이의 가지며 1에 가까울수록 높은 성능을 나타낸다. => $2 \times \frac{Precision \times Recall}{Precision + Recall$

머신러닝에서 풀어야 할 문제의 종류에 따라 모델 성능 평가 방법이 달라지기도 한다.

분류 문제에서의 성능 평가

정확도(Accuracy)

In [8]:
label = [0, 1, 2, 3] # 실제값
predict = [0, 2, 2, 3] # 예측값
# accuracy_score() 메소드로 실제값, 예측값을 인수로 넘겨서 정확도를 계산한다. 인수의 순서는 실제값, 예측값 순서로 지정한다.
# accuracy_score() 메소드의 normalize 옵션의 기본값은 True이고 0부터 1 사이의 값으로 나타낸다.
print(accuracy_score(label, predict))
print(accuracy_score(label, predict, normalize=True))
# normalize 옵션을 False로 지정하면 실제값과 예측값이 일치하는 빈도수를 출력한다.
print(accuracy_score(label, predict, normalize=False))

0.75
0.75
3


혼동 행렬(Confusion Matrix)을 확인하면 실제값과 예측값의 빈도를 행렬 형태로 확인할 수 있다.

In [9]:
label = [2, 0, 2, 2, 0, 1] # 실제값
predict = [0, 0, 2, 2, 0, 2] # 예측값
print(accuracy_score(label, predict))
# confusion_matrix() 메소드로 실제값, 예측값을 인수로 넘겨서 혼동 행렬를 출력한다. 인수의 순서는 실제값, 예측값 순서로 지정한다.
print(confusion_matrix(label, predict))
# 결과를 확인하면 위에서 부터 차례대로 클래스 0, 1, 2를 의미하고 행렬의 행은 실제값을 열은 예측값을 의미한다.
# 즉, 주 대각 원소는 실제값과 예측값이 일치하는 경우를 의미하며, 주 대각 원소가 아닌 원소들은 실제값과 예측값이 차이가
# 나는 경우를 의미한다.

0.6666666666666666
[[2 0 0]
 [0 0 1]
 [1 0 2]]


분류 리포트(Classidication Report)를 확인하면 여러가지 성능 점수를 한 눈에 확인할 수 있다.

In [10]:
label = [2, 0, 2, 2, 0, 1] # 실제값
predict = [0, 0, 2, 2, 0, 2] # 예측값
# classification_report() 메소드로 실제값, 예측값을 인수로 넘겨서 분류 리포트를 출력한다. 인수의 순서는 실제값, 예측값 순서로 지정한다.
print(classification_report(label, predict))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         1
           2       0.67      0.67      0.67         3

    accuracy                           0.67         6
   macro avg       0.44      0.56      0.49         6
weighted avg       0.56      0.67      0.60         6



회귀 문제에서의 성능 평가

MAE(Mean Absolute Error): 실제값과 예측값의 차이의 절대값의 평균을 의미한다.

In [11]:
label = [3, -0.5, 2, 7] # 실제값
predict = [2.5, 0, 2, 8] # 예측값
print(mean_absolute_error(label, predict))
print(mean_absolute_error(predict, label))

0.5
0.5


MSE(Mean Squared Error): 실제값과 예측값의 차이의 제곱값의 평균을 의미한다.  
RMSE(Root Mean Squared Error): 실제값과 예측값의 차이의 제곱값의 평균의 제곱근값을 의미한다.  

In [12]:
label = [3, -0.5, 2, 7] # 실제값
predict = [2.5, 0, 2, 8] # 예측값
print(mean_squared_error(label, predict))
print(mean_squared_error(predict, label))
print(np.sqrt(mean_squared_error(label, predict)))
print(np.sqrt(mean_squared_error(predict, label)))

0.375
0.375
0.6123724356957945
0.6123724356957945


R2 score는 R 제곱값이라고도 부르며 전체 모델에서 설명 가능한 분산의 비율을 나타낸다. 0에서 1 사이의 값을 가지며 1에 가까울수록 높은 성능을 의미한다.

$$R^2 = 1 - \frac{\sum_{i=1}^n(y_i - \hat y)^2}{\sum_{i=1}^n(y_i - \bar y)^2}$$

In [13]:
label = [3, -0.5, 2, 7] # 실제값
predict = [2.5, 0, 2, 8] # 예측값
# r2_score() 메소드로 실제값, 예측값을 인수로 받아 r2 score를 출력한다. 인수의 순서는 실제값, 예측값 순서로 지정한다.
print(r2_score(label, predict))

0.9486081370449679


군집 문제에서의 성능 평가

군집 모델은 비지도 학습을 이용해 생성한 모델을 의미한다.

실루엣 스코어(Silhouette Score)는 서로 다른 군집이 얼마나 잘 분리되어 있는지를 나타내는 지표이다. 같은 군집의 데이터는 가까운 거리에 뭉쳐있고, 다른 군집의 데이터는 멀리 떨어져 있을수록 높은 점수를 나타낸다.  
실루엣 스코어는 아래와 같은 식으로 표현하고 -1부터 1사이의 값을 가지며 점수가 높을수록 좋은 성능을 의미한다.

$$\frac{b - a}{max(a, b)}$$

위 식에서 a는 같은 클래스 내에서의 특정 데이터 포인트와 나머지 클래스 내의 다른 데이터 포인트 간의 평균 거리를 의미하고 b는 특정 데이터 포인트오 두 번째로 가까운 집단 내 데이터 포인트 간의 평균 거리를 의미한다.

In [14]:
x_train = [[1, 2], [4, 5], [2, 1], [6, 7], [2, 3]] # 실제값
label = [0, 1, 0, 1, 0] # 예측값
# silhouette_score() 실제값, 예측값을 인수로 넘겨서 실루엣 스코어를 출력한다. 인수의 순서는 실제값, 예측값 순서로 지정한다.
print(silhouette_score(x_train, label))

0.5789497702625118
